Configuration Parameters

In [163]:
appid = '5ce4119a5a17ae273acc7d0edea6046d'
url = 'https://history.openweathermap.org/data/2.5/history/city'
start = 1633267241 
cnt = 167
csv_url = "citylist.csv"

City list

In [150]:
city_list = [
    {
        'name': 'Milano Malpensa',
        'lat': 45.628008,
        'lon': 8.71227
    },
    {
        'name': 'Milano Linate',
        'lat': 45.460829,
        'lon': 9.278683
    },
    {
        'name': 'Roma Fiumicino ',
        'lat': 41.79478,
        'lon': 12.25477
    },
    {
        'name': 'Roma Ciampino',
        'lat': 41.799133,
        'lon': 12.590576
    },
    {
        'name': 'Olbia ',
        'lat':40.90306,
        'lon':9.515
    },
]

Import and untouchable variables section

In [151]:
import requests
import time as tm
import csv
import pandas as pd
base_payload = {'type': 'hour', 'cnt': cnt, 'appid': appid}

Weather tolist() version ~ 40.5s

In [144]:
start_time = tm.time()

is_first_time = True
for city in city_list:
    time = start
    while time < int(tm.time()):
        payload = base_payload | {'start' : time, 'lat': city['lat'], 'lon': city['lon']}
        data = requests.get(url, params=payload).json()["list"]
        time+= 3600*cnt 
        df = pd.json_normalize(data)
        df = pd.concat([df.drop('weather', axis = 1), pd.DataFrame(df['weather'].str[0].tolist()).add_prefix('weather.')], axis = 1)
        df.insert(0,'city_name',city["name"])
        df.insert(0,'city_lon', city['lat'])
        df.insert(0,'city_lat',city['lon'])
        df.to_csv(csv_url, mode='a', header= is_first_time, index = False)
        is_first_time = True
        
print(tm.time() - start_time)

40.50983428955078


Weather record_path version ~ 39.5s (calling to_csv() for each OWM call)

In [145]:
start_time = tm.time()

is_first_time = True
for city in city_list:
    time = start
    while time < int(tm.time()):
        payload = base_payload | {'start' : time, 'lat': city['lat'], 'lon': city['lon']}
        data = requests.get(url, params=payload).json()["list"]
        time+= 3600*cnt 
        dfw = pd.json_normalize(data, record_path="weather", record_prefix="weather.")
        dfnw = pd.json_normalize(data).drop("weather", axis=1)
        df = pd.concat([dfw, dfnw], axis=1)
        df.insert(0,'city_name',city["name"])
        df.insert(0,'city_lon', city['lat'])
        df.insert(0,'city_lat',city['lon'])
        df.to_csv(csv_url, mode='a', header= is_first_time, index = False)
        is_first_time = False
        
print(tm.time() - start_time)

39.537997007369995


Weather record_path version ~ 38.9s (calling to_csv() for each city)

In [162]:
start_time = tm.time()

is_first_time = True
for city in city_list:
    time = start
    
    df_city= pd.DataFrame()
    while time < int(tm.time()):
        payload = base_payload | {'start' : time, 'lat': city['lat'], 'lon': city['lon']}
        data = requests.get(url, params=payload).json()["list"]
        time+= 3600*cnt 
        dfw = pd.json_normalize(data, record_path="weather", record_prefix="weather.")
        dfnw = pd.json_normalize(data).drop("weather", axis=1)
        df = pd.concat([dfw, dfnw], axis=1)
        df_city = pd.concat([df_city,df], axis=0)
    
    df_city.insert(0,'city_name',city["name"])
    df_city.insert(0,'city_lon', city['lat'])
    df_city.insert(0,'city_lat',city['lon'])
    
    df_city.to_csv(csv_url, mode='a', header= is_first_time, index = False)
    is_first_time = False
        
print(tm.time() - start_time)

38.95713710784912


Weather record_path version ~ 38.7s (calling to_csv() at the end)

In [168]:
start_time = tm.time()

df_cities= pd.DataFrame()
for city in city_list:
    time = start
    
    df_city= pd.DataFrame()
    while time < int(tm.time()):
        payload = base_payload | {'start' : time, 'lat': city['lat'], 'lon': city['lon']}
        data = requests.get(url, params=payload).json()["list"]
        time+= 3600*cnt 
        dfw = pd.json_normalize(data, record_path="weather", record_prefix="weather.")
        dfnw = pd.json_normalize(data).drop("weather", axis=1)
        df = pd.concat([dfw, dfnw], axis=1)
        df_city = pd.concat([df_city,df], axis=0)
        
    df_city.insert(0,'city_name',city["name"])
    df_city.insert(0,'city_lon', city['lat'])
    df_city.insert(0,'city_lat',city['lon'])
    
    df_cities = pd.concat([df_cities,df_city], axis=0)
    
df_cities.to_csv(csv_url, mode='a', header= True, index = False)
        
print(tm.time() - start_time)

38.73245310783386
